In [1]:

# -*- coding: utf-8 -*-
# Bybit & Aster: Recent Trade PnL (1 line = 1 closed trade) + Daily/Symbol Summary
# Python 3.9+
# NOTE:
# - This notebook is designed to run with your API keys from a local `keys.env` file.
# - Internet installation is not performed here. Please ensure the following packages are available:
#     requests, python-dotenv, pybit (>=5.7.0), pandas, numpy
#
# WHAT YOU GET (as DataFrames):
# - df_trades: 1 row = 1 closed trade (per exchange). Columns include times, qty, entry/exit price (if available), net PnL, fees, etc.
# - df_daily:  Daily PnL summary grouped by UTC date and symbol (and exchange). An "ALL" exchange summary is provided too.
#
# HOW THIS WORKS (high level):
# - Bybit: use Unified Trading v5 via pybit to fetch closed PnL list (already "trade till close") and execution list for fees if needed.
# - Aster: use Binance-like signed endpoints (/fapi/v1/income and /fapi/v1/userTrades) to reconstruct "closed trades":
#          we group REALIZED_PNL income entries by order/transaction/time-bucket and enrich them with nearby userTrades to get qty/price.
#
# You can set `SYMBOLS` and `LAST_HOURS` below, then run `run_report(SYMBOLS, LAST_HOURS)`.

import os, time, hmac, hashlib, urllib.parse, requests, math, json
from dataclasses import dataclass, asdict
from typing import Optional, Tuple, List, Dict
from datetime import datetime, timezone, timedelta
from decimal import Decimal, ROUND_FLOOR, ROUND_CEILING

import pandas as pd
import numpy as np
from dotenv import load_dotenv

# If pybit is not installed in your environment, please install it locally before running:
# from pybit.unified_trading import HTTP as BybitHTTP
try:
    from pybit.unified_trading import HTTP as BybitHTTP
except Exception as e:
    BybitHTTP = None
    print("[WARN] pybit.unified_trading not importable. Install pybit>=5.7.0 to enable Bybit calls. Error:", e)

# ================== USER CONFIG ==================
SYMBOLS       = ["ASTERUSDT"]   # e.g., ["BTCUSDT","ETHUSDT","ASTERUSDT"]
LAST_HOURS    = 24              # lookback window
RECV_WINDOW   = 50000
USER_AGENT    = "PnLReporter/1.0"

ASTER_HOST    = "https://fapi.asterdex.com"  # v2 HMAC-ish
ASTER_KEY_HDR = "X-MBX-APIKEY"

# Precision defaults (only used for formatting where needed; not required for reporting)
USE_MANUAL_PRECISION = True
ASTER_PRECISION = {
    "price_tick": Decimal("0.1"),
    "price_precision": 1,
    "qty_step": Decimal("0.001"),
    "qty_min": Decimal("0.001"),
    "qty_precision": 3,
}
BYBIT_PRECISION = {
    "price_tick": Decimal("0.1"),
    "price_precision": 2,
    "qty_step": Decimal("0.001"),
    "qty_min": Decimal("0.001"),
    "qty_precision": 3,
}

# ================== HELPERS ==================
def _now_ms() -> int:
    return int(time.time() * 1000)

def _hmac_sha256(secret: str, msg: str) -> str:
    return hmac.new(secret.encode("utf-8"), msg.encode("utf-8"), hashlib.sha256).hexdigest()

def _round_to_step_dec(val: Decimal, step: Decimal, direction: str) -> Decimal:
    units = (val / step).to_integral_value(rounding=(ROUND_CEILING if direction == "up" else ROUND_FLOOR))
    return units * step

def _limit_decimals_dec(val: Decimal, decimals: int) -> Decimal:
    if decimals <= 0: return Decimal(int(val))
    q = Decimal(1).scaleb(-decimals)
    return val.quantize(q, rounding=ROUND_FLOOR)

# ================== DATA CLASSES ==================
@dataclass
class TradeRow:
    # One row = one closed trade (aggregated at close). Times are UTC strings or None.
    exchange: str         # "ASTER" / "BYBIT"
    symbol: str
    trade_id: str         # best-effort (orderId/positionId/closePnlId/...)
    side: str             # "LONG"/"SHORT" (best-effort) or "BUY"/"SELL"
    qty: float
    entry_price: float
    exit_price: float
    open_time_ms: Optional[int]
    close_time_ms: int
    hold_seconds: Optional[int]
    realized_pnl: float   # without fees if separated
    fee: float            # negative fees (cost). If unknown -> 0.0
    net_pnl: float        # realized_pnl + fee (if realized already includes fees, set fee to 0 and copy realized to net)
    notes: str            # any extra info for debugging

    @property
    def close_time(self) -> str:
        return datetime.fromtimestamp(self.close_time_ms/1000, tz=timezone.utc).strftime("%Y-%m-%d %H:%M:%S")

    @property
    def open_time(self) -> Optional[str]:
        if self.open_time_ms is None: return None
        return datetime.fromtimestamp(self.open_time_ms/1000, tz=timezone.utc).strftime("%Y-%m-%d %H:%M:%S")

# ================== ASTER CLIENT ==================
class AsterClient:
    def __init__(self, api_key: str, api_secret: str, host: str = ASTER_HOST):
        self.api_key = api_key or ""
        self.api_secret = api_secret or ""
        self.host = host

    def _request(self, method: str, path: str, params: dict):
        url = self.host + path
        q = dict(params or {})
        q.setdefault("recvWindow", RECV_WINDOW)
        q["timestamp"] = _now_ms()
        qs = urllib.parse.urlencode(q, doseq=True)
        sig = _hmac_sha256(self.api_secret, qs)
        headers = {ASTER_KEY_HDR: self.api_key, "User-Agent": USER_AGENT}

        if method.upper() == "GET":
            full_url = f"{url}?{qs}&signature={sig}"
            r = requests.get(full_url, headers=headers, timeout=30)
        elif method.upper() == "POST":
            headers["Content-Type"] = "application/x-www-form-urlencoded"
            body = f"{qs}&signature={sig}"
            r = requests.post(url, data=body, headers=headers, timeout=30)
        elif method.upper() == "DELETE":
            headers["Content-Type"] = "application/x-www-form-urlencoded"
            body = f"{qs}&signature={sig}"
            r = requests.delete(url, data=body, headers=headers, timeout=30)
        else:
            raise ValueError("Unsupported method")

        if r.status_code >= 400:
            raise RuntimeError(f"Aster {method} {path} failed: {r.status_code} {r.text[:500]}")
        try:
            return r.json()
        except Exception:
            return r.text

    # ---- History endpoints ----
    def get_income(self, symbol: str = None, start_ms: int = None, end_ms: int = None, limit: int = 1000, incomeType: str = None):
        p = {"limit": limit}
        if symbol:     p["symbol"] = symbol
        if start_ms:   p["startTime"] = start_ms
        if end_ms:     p["endTime"] = end_ms
        if incomeType: p["incomeType"] = incomeType
        return self._request("GET", "/fapi/v1/income", p)

    def get_user_trades(self, symbol: str, start_ms: int = None, end_ms: int = None, limit: int = 1000):
        p = {"symbol": symbol, "limit": limit}
        if start_ms: p["startTime"] = start_ms
        if end_ms:   p["endTime"] = end_ms
        return self._request("GET", "/fapi/v1/userTrades", p)

# ================== BYBIT CLIENT ==================
class BybitClient:
    def __init__(self, api_key: str, api_secret: str, testnet: bool = False):
        if BybitHTTP is None:
            raise RuntimeError("pybit not available. Please install pybit to enable Bybit client.")
        self.session = BybitHTTP(testnet=testnet, api_key=api_key or "", api_secret=api_secret or "")

    def get_closed_pnl_list(self, symbol: str, start_ms: int = None, end_ms: int = None, limit: int = 200):
        params = {"category": "linear", "symbol": symbol, "limit": limit}
        if start_ms: params["startTime"] = start_ms
        if end_ms:   params["endTime"] = end_ms
        r = self.session.get_closed_pnl(**params)
        return (r or {}).get("result", {}).get("list", [])

    def get_execution_list(self, symbol: str, start_ms: int = None, end_ms: int = None, limit: int = 200):
        params = {"category": "linear", "symbol": symbol, "limit": limit}
        if start_ms: params["startTime"] = start_ms
        if end_ms:   params["endTime"] = end_ms
        r = self.session.get_execution_list(**params)
        return (r or {}).get("result", {}).get("list", [])

# ================== NORMALIZATION & RECONSTRUCTION ==================
def _safe_float(*vals, default=0.0):
    for v in vals:
        try:
            return float(v)
        except Exception:
            continue
    return float(default)

def _guess_side_from_pnl(pnl: float) -> str:
    return "LONG" if pnl >= 0 else "SHORT"

def _aster_closed_trades(aster: AsterClient, symbol: str, since_ms: int, time_bucket_ms: int = 5_000) -> List[TradeRow]:
    # Fetch income
    try:
        inc = aster.get_income(symbol=symbol, start_ms=since_ms, limit=1000)
    except Exception as e:
        print("[Aster income] fetch failed:", e)
        inc = []

    # Fetch user trades (fills)
    try:
        uts = aster.get_user_trades(symbol=symbol, start_ms=since_ms, limit=1000)
    except Exception as e:
        print("[Aster userTrades] fetch failed:", e)
        uts = []

    # Build user trade list for nearest lookup
    ut_list = []
    for tr in uts if isinstance(uts, list) else []:
        t_ms = int(tr.get("time") or tr.get("transactTime") or tr.get("T") or 0)
        ut_list.append({
            "t": t_ms,
            "side": str(tr.get("side") or tr.get("S") or "").upper(),
            "qty": _safe_float(tr.get("qty"), tr.get("executedQty"), tr.get("q"), default=0.0),
            "price": _safe_float(tr.get("price"), tr.get("p"), default=0.0),
            "orderId": str(tr.get("orderId") or tr.get("orderID") or ""),
        })
    ut_list.sort(key=lambda x: x["t"])

    def nearest_trade(ts):
        best = None; best_dt = 1e18
        for row in ut_list:
            dt = abs(row["t"] - ts)
            if dt < best_dt:
                best_dt = dt; best = row
        return best

    # Group income by likely close identifier
    groups: Dict[str, Dict] = {}

    def add_to_group(key: str, *, ts: int, pnl: float = 0.0, fee: float = 0.0):
        g = groups.setdefault(key, {"ts": ts, "pnl": 0.0, "fee": 0.0})
        g["ts"] = max(g["ts"], ts)
        g["pnl"] += pnl
        g["fee"] += fee

    for it in inc if isinstance(inc, list) else []:
        ts = int(it.get("time") or it.get("tranTime") or it.get("timestamp") or 0)
        typ = str(it.get("incomeType") or it.get("type") or "")
        val = _safe_float(it.get("income"), it.get("amount"), default=0.0)

        raw_oid = str(it.get("orderId") or it.get("orderID") or it.get("tranId") or "")
        if raw_oid and raw_oid != "0":
            key = f"oid:{raw_oid}"
        else:
            bucket = (ts // time_bucket_ms) * time_bucket_ms
            key = f"t:{bucket}"

        if "REALIZED" in typ.upper() or "PNL" in typ.upper():
            add_to_group(key, ts=ts, pnl=val, fee=0.0)
        elif "COMMISSION" in typ.upper() or "FEE" in typ.upper():
            add_to_group(key, ts=ts, pnl=0.0, fee=val)
        else:
            pass

    out: List[TradeRow] = []
    for key, v in groups.items():
        ts = int(v["ts"])
        pnl = float(v["pnl"])
        fee = float(v["fee"])

        ut = nearest_trade(ts) or {}
        side_raw = str(ut.get("side", "")).upper()
        side = ("LONG" if side_raw == "BUY" else ("SHORT" if side_raw == "SELL" else _guess_side_from_pnl(pnl)))
        qty = float(ut.get("qty") or 0.0)
        price = float(ut.get("price") or 0.0)

        out.append(TradeRow(
            exchange="ASTER",
            symbol=symbol,
            trade_id=key,
            side=side,
            qty=qty,
            entry_price=float("nan"),
            exit_price=price if price else float("nan"),
            open_time_ms=None,
            close_time_ms=ts,
            hold_seconds=None,
            realized_pnl=pnl,
            fee=fee,
            net_pnl=pnl + fee,
            notes="aster_recon_via_income_userTrades"
        ))

    return sorted(out, key=lambda r: r.close_time_ms)

def _bybit_closed_trades(bybit: BybitClient, symbol: str, since_ms: int) -> List[TradeRow]:
    try:
        cl = bybit.get_closed_pnl_list(symbol=symbol, start_ms=since_ms, limit=200)
    except Exception as e:
        print("[Bybit closed_pnl] fetch failed:", e)
        cl = []

    try:
        exes = bybit.get_execution_list(symbol=symbol, start_ms=since_ms, limit=500)
    except Exception as e:
        print("[Bybit execution] fetch failed:", e)
        exes = []

    exec_rows = []
    for r in exes if isinstance(exes, list) else []:
        t_ms = int(str(r.get("execTimeNs") or "0")[:13]) if r.get("execTimeNs") else int(r.get("execTime") or 0)
        exec_rows.append({
            "t": t_ms,
            "orderId": str(r.get("orderId") or ""),
            "fee": _safe_float(r.get("execFee"), default=0.0) * (-1.0),
        })
    exec_rows.sort(key=lambda x: x["t"])

    def nearest_fee(ts, order_id):
        best = None; best_dt = 1e18
        fee_sum = 0.0
        if order_id:
            for e in exec_rows:
                if e["orderId"] == order_id:
                    fee_sum += e["fee"]
            if fee_sum != 0.0:
                return fee_sum
        for e in exec_rows:
            dt = abs(e["t"] - ts)
            if dt < best_dt:
                best_dt = dt; best = e
        return (best or {}).get("fee", 0.0)

    out: List[TradeRow] = []
    for row in cl if isinstance(cl, list) else []:
        close_ts = int(row.get("updatedTime") or row.get("createdTime") or row.get("time") or 0)
        trade_id = str(row.get("orderId") or row.get("orderID") or row.get("id") or f"t:{close_ts}")
        side_raw = str(row.get("side") or "").upper()
        side = "LONG" if side_raw == "BUY" else ("SHORT" if side_raw == "SELL" else _guess_side_from_pnl(_safe_float(row.get('closedPnl'), row.get('realisedPnl'))))
        qty  = _safe_float(row.get("qty"), row.get("size"), row.get("closedSize"))
        entry = _safe_float(row.get("avgEntryPrice"), row.get("entryPrice"))
        exitp = _safe_float(row.get("avgExitPrice"), row.get("exitPrice"))
        open_ts = int(row.get("createdTime") or 0)
        hold_s = int((close_ts - open_ts)/1000) if open_ts and close_ts else None
        realized = _safe_float(row.get("closedPnl"), row.get("realisedPnl"), row.get("pnl"))
        fee = _safe_float(row.get("fees"), default=0.0)
        if fee == 0.0:
            fee = nearest_fee(close_ts, trade_id)

        out.append(TradeRow(
            exchange="BYBIT",
            symbol=symbol,
            trade_id=trade_id,
            side=side,
            qty=qty,
            entry_price=entry if entry else float("nan"),
            exit_price=exitp if exitp else float("nan"),
            open_time_ms=open_ts if open_ts else None,
            close_time_ms=close_ts,
            hold_seconds=hold_s,
            realized_pnl=realized,
            fee=fee,
            net_pnl=realized + fee,
            notes="bybit_closed_pnl"
        ))

    return sorted(out, key=lambda r: r.close_time_ms)

# ================== REPORT PIPELINE ==================
def fetch_trades_for_symbols(symbols: List[str], last_hours: int,
                             aster: Optional[AsterClient],
                             bybit: Optional[BybitClient]) -> List[TradeRow]:
    since_ms = int(time.time()*1000) - last_hours*60*60*1000
    rows: List[TradeRow] = []
    for sym in symbols:
        if aster:
            try:
                rows.extend(_aster_closed_trades(aster, sym, since_ms))
            except Exception as e:
                print(f"[ASTER {sym}] error:", e)
        if bybit:
            try:
                rows.extend(_bybit_closed_trades(bybit, sym, since_ms))
            except Exception as e:
                print(f"[BYBIT {sym}] error:", e)
    return rows

def as_dataframe(trades: List[TradeRow]) -> pd.DataFrame:
    if not trades:
        cols = ["exchange","symbol","trade_id","side","qty","entry_price","exit_price",
                "open_time","close_time","hold_seconds","realized_pnl","fee","net_pnl","notes"]
        return pd.DataFrame(columns=cols)
    df = pd.DataFrame([{
        "exchange": t.exchange,
        "symbol": t.symbol,
        "trade_id": t.trade_id,
        "side": t.side,
        "qty": t.qty,
        "entry_price": t.entry_price,
        "exit_price": t.exit_price,
        "open_time": t.open_time,
        "close_time": t.close_time,
        "hold_seconds": t.hold_seconds,
        "realized_pnl": t.realized_pnl,
        "fee": t.fee,
        "net_pnl": t.net_pnl,
        "notes": t.notes,
    } for t in trades])
    return df

def daily_symbol_summary(df_trades: pd.DataFrame) -> pd.DataFrame:
    # 日付(UTC)/銘柄ごとのPLサマリー（取引所別）＋ Exchange="ALL" の合算行
    if df_trades.empty:
        cols = ["date_utc","symbol","exchange","trades","wins","losses","win_rate","gross_profit","gross_loss","net_pnl","avg_per_trade"]
        return pd.DataFrame(columns=cols)

    df = df_trades.copy()
    df["date_utc"] = pd.to_datetime(df["close_time"]).dt.tz_localize("UTC").dt.date
    def _agg(g):
        pnls = g["net_pnl"].astype(float)
        wins = (pnls > 0).sum()
        losses = (pnls < 0).sum()
        gross_p = pnls[ pnls > 0].sum()
        gross_l = pnls[ pnls < 0].sum()
        return pd.Series({
            "trades": len(g),
            "wins": int(wins),
            "losses": int(losses),
            "win_rate": (wins/len(g)) if len(g) else 0.0,
            "gross_profit": gross_p,
            "gross_loss": gross_l,
            "net_pnl": pnls.sum(),
            "avg_per_trade": pnls.mean() if len(g) else 0.0,
        })

    g_ex = df.groupby(["date_utc","symbol","exchange"], as_index=False).apply(_agg).reset_index().drop(columns=["level_0"])
    g_all = df.groupby(["date_utc","symbol"], as_index=False).apply(_agg).reset_index().drop(columns=["level_0"])
    g_all["exchange"] = "ALL"
    g_ex = g_ex[["date_utc","symbol","exchange","trades","wins","losses","win_rate","gross_profit","gross_loss","net_pnl","avg_per_trade"]]
    g_all = g_all[["date_utc","symbol","exchange","trades","wins","losses","win_rate","gross_profit","gross_loss","net_pnl","avg_per_trade"]]
    out = pd.concat([g_ex, g_all], ignore_index=True).sort_values(["date_utc","symbol","exchange"])
    return out

# ================== MAIN ENTRY ==================
def run_report(symbols: List[str] = None, last_hours: int = LAST_HOURS):
    symbols = symbols or SYMBOLS

    load_dotenv("keys.env")
    aster_key    = os.getenv("ASTER_API_KEY") or ""
    aster_secret = os.getenv("ASTER_API_SECRET") or ""
    bybit_key    = os.getenv("BYBIT_API_KEY") or ""
    bybit_secret = os.getenv("BYBIT_API_SECRET") or ""
    bybit_testnet = (os.getenv("BYBIT_TESTNET","false").lower() == "true")

    aster = None
    bybit = None
    if aster_key and aster_secret:
        aster = AsterClient(aster_key, aster_secret, host=ASTER_HOST)
    else:
        print("[INFO] ASTER keys missing; skipping Aster.")
    if bybit_key and bybit_secret and (BybitHTTP is not None):
        bybit = BybitClient(bybit_key, bybit_secret, testnet=bybit_testnet)
    else:
        if BybitHTTP is None:
            print("[INFO] pybit not available; skipping Bybit.")
        else:
            print("[INFO] BYBIT keys missing; skipping Bybit.")

    trades = fetch_trades_for_symbols(symbols, last_hours, aster, bybit)
    df_trades = as_dataframe(trades)
    df_daily  = daily_symbol_summary(df_trades)

    return df_trades, df_daily

# === Example (uncomment to run locally) ===
df_trades, df_daily = run_report(SYMBOLS, LAST_HOURS)
display(df_trades.head(50))
display(df_daily.head(50))


[Bybit execution] fetch failed: 'HTTP' object has no attribute 'get_execution_list'


KeyError: "['level_0'] not found in axis"